# Environment setup

## Google Drive mount
I'm using Google Colaboratory as my default platform, therefore I need to set up my environment to integrate it with Google Drive. You can skip this bit if you're working locally.

1. Mount Google Drive on the runtime to be able to read and write files. This will ask you to log in to your Google Account and provide an authorization code.
2. Create a symbolic link to a working directory 
3. Change the directory to the one where I cloned my repository.


In [1]:
# mount Google Drive on the runtime
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
# create a symbolic link to a working directory
!ln -s /content/gdrive/My\ Drive/Colab\ Notebooks/land-cover-classification /mydrive

# navigate to the working directory
%cd /mydrive

/content/gdrive/My Drive/Colab Notebooks/land-cover-classification


## Modules installation

In [6]:
!pip install -qq OWSLib
!pip install -qq fastai

In [7]:
from owslib.wms import WebMapService
import numpy as np
import tqdm.notebook as tq
from fastai.vision.core import PILImage, PILMask
from PIL import Image

ModuleNotFoundError: ignored

https://www.geoportal.gov.pl/uslugi/usluga-przegladania-wms

https://geopython.github.io/OWSLib/usage.html#wms

https://zieniewicz-m-1992.medium.com/

https://www.geoportal.gov.pl/uslugi/usluga-przegladania-wms#

https://www.esa.int/Applications/Observing_the_Earth/Copernicus/Sentinel-2/Land-cover_maps_of_Europe_from_the_Cloud

http://s2glc.cbk.waw.pl/extension

https://creodias.eu/wms-request

https://www.kaggle.com/dipam7/image-segmentation-using-fastai

https://www.kaggle.com/alt250/multi-label-segmentation-using-fastai

https://www.kaggle.com/balraj98/deepglobe-land-cover-classification-deeplabv3

# Data retrieval

In [5]:
import os
class CustomWMS:

  def __init__(self, url, version):
    self.url = url
    self.version = version
    self.wms = WebMapService(self.url, self.version)

  def available_layers(self):
    self.layers = list(self.wms.contents)
    return self.layers

  def choose_layer(self, layer):
    self.layer = layer
    self.title = self.wms[layer].title    
    self.extents = self.wms[layer].boundingBox
    self.crs = self.wms[layer].crsOptions
    self.styles = self.wms[layer].styles
    self.formats = self.wms.getOperationByName('GetMap').formatOptions

  def save_image(self, bbox, srs, style='default', size=(512, 512), format='image/png', trans=False, out='output'):
    img = self.wms.getmap(        
        bbox=bbox,
        srs=srs,
        layers=[self.layer],
        styles=[style],
        size=size,
        format=format,
        transparent=trans
        )
    
    os.makedirs('output', exist_ok=True)

    with open(f'{out}/{self.title}.png', 'wb') as out:
      out.write(img.read())

In [23]:
msk = PILMask.create()
msk.show(figsize=(5,5), alpha=1)

ModuleNotFoundError: ignored

In [6]:
orto = CustomWMS(
    'https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/StandardResolution',
    '1.1.1'
)

orto.choose_layer('Raster')

orto.save_image(
    bbox=(20, 52, 20.01, 52.01),
    srs='EPSG:4326'    
)

In [7]:
cover = CustomWMS(
    'http://integracja.gugik.gov.pl/cgi-bin/arimr_pokrycie',
    '1.3.0'
)

cover.choose_layer('pow')

cover.save_image(
    bbox=(20, 52, 20.01, 52.01),
    srs='EPSG:4326'    
)

## Scraping

In [10]:
orto = WebMapService(
    'https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/StandardResolution',
    '1.1.1'
)

cover = WebMapService(
    'http://integracja.gugik.gov.pl/cgi-bin/arimr_pokrycie',
    '1.3.0'
)

# land = WebMapService(
#     'http://mapy.geoportal.gov.pl/wss/geosrv/wmsLC/wms',
#     '1.1.1'
# )

In [14]:
land = WebMapService(
    'http://services.sentinel-hub.com/ogc/wms/',
    '1.1.1'
)

HTTPError: ignored

In [12]:
land.contents

OrderedDict([('LU.ZoningElement',
              <owslib.map.wms130.ContentMetadata at 0x7fb6907abc50>),
             ('LU.SupplementaryRegulation_polygon',
              <owslib.map.wms130.ContentMetadata at 0x7fb6907b1110>),
             ('LU.SupplementaryRegulation_line',
              <owslib.map.wms130.ContentMetadata at 0x7fb6907b1510>),
             ('LU.SupplementaryRegulation_point',
              <owslib.map.wms130.ContentMetadata at 0x7fb6907b18d0>),
             ('LU.SpatialPlan',
              <owslib.map.wms130.ContentMetadata at 0x7fb6907b1c90>)])

In [ ]:
land.contents
land['LandCover'].boundingBox

    # self.layer = layer
    # self.title = self.wms[layer].title    
    # self.extents = self.wms[layer].boundingBox
    # self.crs = self.wms[layer].crsOptions
    # self.styles = self.wms[layer].styles
    # self.formats = self.wms.getOperationByName('GetMap').formatOptions

(160828.0, 98928.9, 876030.0, 796522.0, 'EPSG:2180')

In [ ]:
# from pyproj import Proj, transform

# inProj = Proj(init='epsg:2180')
# outProj = Proj(init='epsg:4326')
# x1,y1 = 160828.0,98928.9
# x2,y2 = transform(inProj,outProj,x1,y1)
# x2,y2

from pyproj import Transformer

transformer = Transformer.from_crs('EPSG:2180', 'EPSG:4326')
transformer.transform(160828.0, 98928.9)

(49.18351092325662, 13.4934453041342)

In [ ]:
xmin, ymin, xmax, ymax = orto.extents[:4]
xmin, ymin, xmax, ymax

(14.0, 48.901811, 24.759301, 54.925806)

In [ ]:
for n in tq.tqdm(range(20)):

  x_centroid = np.random.uniform(xmin, xmax)
  y_centroid = np.random.uniform(ymin, ymax)
  box_size = 0.005
  bbox = (
      x_centroid-box_size,
      y_centroid-box_size,
      x_centroid+box_size,
      y_centroid+box_size
      )

  os.makedirs('output', exist_ok=True)

  orto_img = orto.getmap(        
          bbox=bbox,
          srs='EPSG:4326',
          layers=['Raster'],
          styles=['default'],
          size=(1024,1024),
          format='image/jpeg',
          # transparent=False
          )
  
  with open(f'output/{n}_orto_{bbox[0]:.6f}-{bbox[1]:.6f}-{bbox[2]:.6f}-{bbox[3]:.6f}.jpg', 'wb') as out:
    out.write(orto_img.read())

  cover_img = cover.getmap(        
          bbox=bbox,
          srs='EPSG:4326',
          layers=['pow'],
          styles=['default'],
          size=(1024,1024),
          format='image/jpeg',
          # transparent=False
          )
  
  with open(f'output/{n}_cover_{bbox[0]:.6f}-{bbox[1]:.6f}-{bbox[2]:.6f}-{bbox[3]:.6f}.jpg', 'wb') as out:
    out.write(cover_img.read())
  
  land_img = land.getmap(        
          bbox=bbox,
          srs='EPSG:4326',
          layers=['LC.LandCoverSurfaces'],
          styles=['default'],
          size=(1024,1024),
          format='image/jpeg',
          # transparent=False
          )
  
  with open(f'output/{n}_land_{bbox[0]:.6f}-{bbox[1]:.6f}-{bbox[2]:.6f}-{bbox[3]:.6f}.jpg', 'wb') as out:
    out.write(land_img.read())

  
  





ConnectTimeout: ignored

# Old

In [ ]:


img = wms.getmap(
    layers=['pow'],
    # styles=['default'],
    srs='EPSG:4326',
    # bbox=(14.076638, 48.980459, 24.551068, 54.668798), #minx miny maxx maxy
    bbox=(20, 52, 20.01, 52.01),
    # 14.076638, 48.980459, 24.551068, 54.668798
    size=(1000, 1000),
    format='image/png',
    # transparent=True

In [ ]:
orto.choose_layer('Raster')

In [ ]:
# map_url = 'https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/HighResolution'
# map_url = 'https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/StandardResolution'
# map_url = 'https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/TrueOrtho'
map_url = 'http://integracja.gugik.gov.pl/cgi-bin/arimr_pokrycie'

# wms = WebMapService(map_url, version='1.1.1')
wms = WebMapService(map_url, version='1.3.0')
# available layers
wms.contents

OrderedDict([('pow', <owslib.map.wms130.ContentMetadata at 0x7f6b8c837e10>),
             ('lin', <owslib.map.wms130.ContentMetadata at 0x7f6b8c837d90>),
             ('pkt', <owslib.map.wms130.ContentMetadata at 0x7f6baf22a290>)])

In [ ]:
# titlewms['Raster'].title

KeyError: ignored

In [ ]:
# queryable
wms['Raster'].queryable

In [ ]:
# opaque
wms['Raster'].opaque

In [ ]:
# bounding box
wms['Raster'].boundingBox

(160828.0, 98928.9, 876030.0, 796522.0, 'EPSG:2180')

In [ ]:
# styles
wms['pow'].styles

{'default': {'legend': 'http://integracja.gugik.gov.pl/cgi-bin/arimr_pokrycie?version=1.3.0&service=WMS&request=GetLegendGraphic&sld_version=1.1.0&layer=pow-02&format=image/png&STYLE=default',
  'legend_format': 'image/png',
  'legend_height': '600',
  'legend_width': '800',
  'title': 'default'}}

In [ ]:
# crs
wms['pow'].crsOptions

['EPSG:2178', 'EPSG:2179', 'EPSG:2176', 'EPSG:2177', 'EPSG:2180', 'EPSG:4326']

In [ ]:
[op.name for op in wms.operations]

['GetCapabilities',
 'GetMap',
 'GetFeatureInfo',
 'DescribeLayer',
 'GetLegendGraphic',
 'GetStyles']

In [ ]:
wms.getOperationByName('GetMap').methods

[{'type': 'Get',
  'url': 'http://integracja.gugik.gov.pl/cgi-bin/arimr_pokrycie?'},
 {'type': 'Post',
  'url': 'http://integracja.gugik.gov.pl/cgi-bin/arimr_pokrycie?'}]

In [ ]:
wms.getOperationByName('GetMap').formatOptions

['image/png',
 'image/jpeg',
 'image/png; mode=8bit',
 'image/tiff',
 'application/vnd.google-earth.kml+xml',
 'application/vnd.google-earth.kmz']

https://pl.wikipedia.org/wiki/Web_Map_Service#GetMap

https://okmaps.org/ogi/WMS.aspx#GM

https://www.spatialreference.org/ref/epsg/2180/

https://www.spatialreference.org/ref/epsg/2180/

In [ ]:
img = wms.getmap(
    layers=['pow'],
    # styles=['default'],
    srs='EPSG:4326',
    # bbox=(14.076638, 48.980459, 24.551068, 54.668798), #minx miny maxx maxy
    bbox=(20, 52, 20.01, 52.01),
    # 14.076638, 48.980459, 24.551068, 54.668798
    size=(1000, 1000),
    format='image/png',
    # transparent=True
    )
with open('pokr.png', 'wb') as out:
  out.write(img.read())

In [ ]:

# service=WMS&
# version=1.3.0&
# request=GetMap&
# layers=pow&
# styles=&
# width=1000&height=1000&
# crs=EPSG:4326&
# bbox=48.980459,14.076638,54.668798,24.551068&
# format=image/jpeg&
# transparent=FALSE&
# bgcolor=0xFFFFFF&exceptions=XML

In [ ]:
img.geturl()

'http://integracja.gugik.gov.pl/cgi-bin/arimr_pokrycie?service=WMS&version=1.3.0&request=GetMap&layers=pow&styles=default&width=1000&height=1000&crs=EPSG:4326&bbox=48.980459,14.076638,54.668798,24.551068&format=image/png&transparent=FALSE&bgcolor=0xFFFFFF&exceptions=XML'